In [ ]:
class CMA_ES:
    def __init__(self, x0, sigma, maxfevals = 10000, popsize = None, weights = None):
        N = x0.shape[0]
        self.dimension = N
        self.chiN = N**0.5 * (1 - 1. / (4 * N) + 1. / (21 * N**2))
        self.lam = 4 + int(3 * np.log(N)) if not popsize else popsize
        print(f"Popsize: {self.lam}")
        self.mu = int(self.lam / 2)
        
        if weights:
            self.weights = weights
        else:
            self.weights = np.array([np.log(self.lam / 2 + 0.5) - np.log(i + 1) if i < self.mu else 0
                        for i in range(self.lam)])
            self.weights /= np.sum(self.weights)
        self.mueff = np.sum(self.weights)**2 / np.sum(self.weights**2)
        
        self.cc = (4 + self.mueff/N) / (N+4 + 2 * self.mueff/N)
        self.cs = (self.mueff + 2) / (N + self.mueff + 5)
        self.c1 = 2 / ((N + 1.3)**2 + self.mueff) 
        self.cmu = min([1 - self.c1, 2 * (self.mueff - 2 + 1/self.mueff) / ((N + 2)**2 + self.mueff)])
        self.damps = 2 * self.mueff/self.lam + 0.3 + self.cs

        self.xmean = x0[:]
        self.sigma = sigma
        self.pc = np.zeros(N) 
        self.ps = np.zeros(N) 
        self.lazy_gap_evals = 0.5 * N * self.lam * (self.c1 + self.cmu)**-1 / N**2
        self.maxfevals = maxfevals
        self.C = np.identity(N)
        self.counteval = 0 
        self.fitvals = []   
        self.best = (x0, None)
        self.condition_number = 1
        self.eigen_values = np.ones(N)
        self.eigen_vectors = np.identity(N)
        self.updated_eval = 0
        self.inv_sqrt = np.ones(N)

    def _update_eigensystem(self, current_eval, lazy_gap_evals):
        if current_eval <= self.updated_eval + lazy_gap_evals:
            return self
        self.eigen_values, self.eigen_vectors = np.linalg.eig(self.C)
        self.inv_sqrt = self.eigen_vectors @ np.diag(self.eigen_values**-0.5) @ self.eigen_vectors.T
        self.condition_number = self.eigen_values.max() / self.eigen_values.min()
         
    def sample(self):
        """Wylosuj próbkę nowych osobników"""
        self._update_eigensystem(self.counteval, self.lazy_gap_evals)
        y = np.random.multivariate_normal(np.zeros(self.dimension), self.C, self.lam)
        y = self.xmean + self.sigma * y
        return y
    
    def update(self, x, fitvals):
        """Zaktualizuj wartości uzyskanych parametrów"""
        self.counteval += fitvals.shape[0]  # Zwiększamy licznik wykonań
        N = self.xmean.shape[0]
        x_old = self.xmean.copy()
        
        # Posortuj osobniki po wartości funkcji celu
        sorted_indices = np.argsort(fitvals)
        x = x[sorted_indices]
        self.fitvals = fitvals[sorted_indices]
        self.best = (x[0], self.fitvals[0])
        
        # Aktualizacja średniej
        self.xmean = (self.weights @ x).ravel()
        y = (x - x_old) / self.sigma
        y_w = (self.weights @ y).ravel()
        z = self.inv_sqrt @ y_w
    
        # Aktualizacja ścieżek ewolucji
        chiN = np.sqrt(N) * (1 - 1. / (4 * N) + 1. / (21 * N**2))
        hsig = np.linalg.norm(self.ps) / np.sqrt(1 - (1 - self.cs)**(2 * self.counteval / self.lam)) < (1.4 + 2 / (N + 1) * chiN)
        self.pc = (1 - self.cc) * self.pc + hsig * np.sqrt(self.cc * (2 - self.cc) * self.mueff) * z
        self.ps = (1 - self.cs) * self.ps + np.sqrt(self.cs * (2 - self.cs) * self.mueff) * y_w
        
        # Wzór dla aktywnej modyfikacji CMA-ES
        # Zaktualizowana wersja macierzy Z
        Z = np.zeros_like(self.C)
        w_pos = self.weights[self.weights > 0]
        w_neg = self.weights[self.weights < 0]
        
        Z = self.inv_sqrt @ (np.mean(x[:self.mu], axis=0) - np.mean(x[self.mu:], axis=0)).T

        # Nowa wartość dla ccov i beta
        ccov = 2 / ((N + np.sqrt(2))**2)
        beta = 4 * self.mu - 2 / ((N + 12)**2 + 4 * self.mu)

        # Aktualizacja macierzy kowariancji
        rank_one_update = self.c1 * np.outer(self.pc, self.pc)
        rank_mu_update = self.cmu * np.sum([
            w_pos[i] * np.outer(y[i], y[i]) for i in range(self.mu)
        ], axis=0)

        self.C = (1 - ccov) * self.C + ccov * (rank_one_update + rank_mu_update + beta * Z)
        self.C = (self.C + self.C.T) / 2.0  # Upewniamy się, że macierz jest symetryczna
        
        # Aktualizacja rozmiaru kroku
        self.sigma *= np.exp(min(1, (self.cs / self.damps) * (np.linalg.norm(self.ps) / self.chiN - 1)))

    def terminate(self):
        """Zakończ algorytm"""
        if self.counteval <= 0:
            return False
        if self.counteval >= self.maxfevals:
            return True
        if self.condition_number > 1e13:
            return True
        if self.sigma * np.max(self.eigen_values)**0.5 < 1e-13:
            return True
        return False
